<a href="https://colab.research.google.com/github/Jack3690/pydol/blob/main/notebooks/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/Jack3690/pydol/ -q
!pip install astroquery -q

  Preparing metadata (setup.py) ... done


# **DOLPHOT Installation**

In [2]:
!wget http://americano.dolphinsim.com/dolphot/dolphot2.0.tar.gz
!wget http://americano.dolphinsim.com/dolphot/dolphot2.0.NIRCAM.tar.gz
!wget http://americano.dolphinsim.com/dolphot/nircam_F140M.tar.gz
!wget http://americano.dolphinsim.com/dolphot/nircam_F212N.tar.gz

--2024-06-29 15:56:45--  http://americano.dolphinsim.com/dolphot/dolphot2.0.tar.gz
Resolving americano.dolphinsim.com (americano.dolphinsim.com)... 64.139.89.252
Connecting to americano.dolphinsim.com (americano.dolphinsim.com)|64.139.89.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 270579 (264K) [application/x-gzip]
Saving to: ‘dolphot2.0.tar.gz’

dolphot2.0.tar.gz   100%[===================>] 264.24K   245KB/s    in 1.1s    

2024-06-29 15:56:47 (245 KB/s) - ‘dolphot2.0.tar.gz’ saved [270579/270579]

--2024-06-29 15:56:47--  http://americano.dolphinsim.com/dolphot/dolphot2.0.NIRCAM.tar.gz
Resolving americano.dolphinsim.com (americano.dolphinsim.com)... 64.139.89.252
Connecting to americano.dolphinsim.com (americano.dolphinsim.com)|64.139.89.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82691 (81K) [application/x-gzip]
Saving to: ‘dolphot2.0.NIRCAM.tar.gz’

dolphot2.0.NIRCAM.t 100%[===================>]  80.75K   193KB/s   

In [3]:
!tar -xf dolphot2.0.tar.gz
!tar -xf dolphot2.0.NIRCAM.tar.gz
!tar -xf nircam_F140M.tar.gz
!tar -xf nircam_F212N.tar.gz

In [4]:
with open('/content/dolphot2.0/Makefile') as f:
  dat = f.readlines()
dat[28]= dat[28][1:]

with open('/content/dolphot2.0/Makefile','w') as f:
  f.writelines(dat)

In [5]:
cd /content/dolphot2.0/

/content/dolphot2.0


In [6]:
import os

In [7]:
os.system('make all')

0

In [8]:
cd /content/

/content


In [10]:
if 'dolphot2.0' not in os.environ['PATH']:
  os.environ['PATH'] += ':/content/dolphot2.0/bin'
  os.environ['PATH'] += ':/content/dolphot2.0/nircam'

# **Imports**

In [2]:
from glob import glob
import astropy.io.fits as fits

import os
import astroquery
from astroquery.mast import Observations
import pandas as pd

from astropy.table import Table
from astropy.wcs import WCS
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import json
from astropy.coordinates import angular_separation
import astropy.units as u
from pydol.pipeline.jwst import jpipe
import multiprocessing as mp

# **Data**

**Observation ID format**

jw\<ppppp>\<ooo><vvv>\_\<gg>\<s>\<aa>\_\<eeeee>(-<"seg"NNN>)\_\<detector>\_\<prodType>.fits

where

ppppp: program ID number

ooo: observation number

vvv: visit number

gg: visit group

s: parallel sequence ID (1=prime, 2-5=parallel)

aa: activity number (base 36)

eeeee: exposure number

segNNN: the text “seg” followed by a three-digit segment number (optional)

detector: detector name (e.g. ‘nrca1’, ‘nrcblong’, ‘mirimage’)

prodType: product type identifier (e.g. ‘uncal’, ‘rate’, ‘cal’)

In [ ]:
#Example data for F212N filter for M82

mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '/content/data/stage0/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

ext = 'uncal'
# JWST images to be analyzed
image_files = []

for i in range(1,10):
    for j in range(1,5):
            for l in ['b']:
                image_files.append( f"jw01701052001_02102_0000{i}_nrc{l}{j}_uncal.fits")

for i in range(10,17):
    for j in range(1,5):
            for l in ['b']:
                image_files.append( f"jw01701052001_02102_000{i}_nrc{l}{j}_uncal.fits")

for image_file in image_files:
    # Download file (if not already downloaded)
    mast_path  = os.path.join(mast_dir, image_file)
    local_path = os.path.join(data_dir, image_file)
    Observations.download_file(mast_path, local_path=local_path)

In [12]:
#Example data for M82

mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '/content/photometry/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

# JWST images to be analyzed
image_files = [ f"jw01701-o052_t007_nircam_clear-f212n-sub640_i2d.fits"]

for image_file in image_files:
    # Download file (if not already downloaded)
    mast_path  = os.path.join(mast_dir, image_file)
    local_path = os.path.join(data_dir, image_file)
    Observations.download_file(mast_path, local_path=local_path)

# **Pipeline**

In [ ]:
# For reducing load on the RAM. Let's split pipeline into two steps

input_files = glob(f'/content/data/stage0/*')[:32]
out_dir = '/content/'
jwst_data = jpipe(input_files, out_dir)
jwst_data()

input_files = glob(f'/content/data/stage0/*')[32:]
out_dir = '/content/'
jwst_data = jpipe(input_files, out_dir)
jwst_data()

# **DOLPHOT Photometry**

In [14]:
if 'dolphot2.0' not in os.environ['PATH']:
  os.environ['PATH'] += ':/content/dolphot2.0/bin'
  os.environ['PATH'] += ':/content/dolphot2.0/nircam'

In [6]:
cal_files = glob(f'/content/data/stage2/*b1*cal.fits')

In [7]:
from pydol.photometry import nircam

In [ ]:
nircam.nircam_phot(cal_files,
                   filter='f212n',
                   output_dir='/content/photometry/',
                   drz_path='/content/photometry/jw01701-o052_t007_nircam_clear-f212n-sub640_i2d',
                   cat_name= 'b1')